# Проект по классификации комментариев с BERT

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Критерий качества модели: *F1* не меньше 0.75.

**Цель проекта:** создать модель, предсказывающую токсичность комментариев, с метрикой качества F1 не меньше 0.75.

**Задачи:**
1. Изучить предоставленные данные.
2. Проверить данные на дисбаланс классов.
3. Предобработать данные (токенизация, создание эмбеддингов)
4. Обучить несколько моделей классификации и выбрать наилучшую для предсказания на тестовых данных.

**Содержание**

<a href='#section1'>1 Подготовка</a>

<a href='#section1.1'>1.1 Установка библиотек</a>

<a href='#section1.2'>1.2 Загрузка и анализ данных</a>

<a href='#section1.3'>1.3 Предобработка данных</a>

<a href='#section2'>2 Обучение</a>

<a href='#section3'>3 Выводы</a>

<a id='section1'></a>
## Подготовка

<a id='section1.1'></a>
### Установка библиотек

Установим необходимые для работы библиотеки и подключим Google Drive.

In [3]:
# установка обновлений
!pip install scikit-learn --upgrade -q
!pip install matplotlib --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 54.2 MB/s eta 0:00:00


In [4]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers

In [5]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import (
    f1_score,
    make_scorer
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from tqdm import notebook

from transformers import (AutoTokenizer,
                          AutoModel)

from google.colab import drive
from google.colab import files

In [6]:
# константа
RANDOM_STATE = 42

<a id='section1.2'></a>
### Загрузка и анализ данных

Загрузим и изучим данные.

In [7]:
try:
    # путь для платформы Яндекс Практикум
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

except:
    # подключаем Google Drive
    drive.mount('/content/drive')

    # путь для Гугл Колаб
    toxic_comments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Files/Portfolio/toxic_comments.csv')

Mounted at /content/drive


Выведем всю необходимую информацию о датасете на экран.

In [8]:
# первые 5 строк датасета
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


В данных есть колонка `Unnamed: 0`, о которой не сообщалось в ТЗ.

In [9]:
# общая информация
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Формат данных ко всех колонках корректный. Пропусков нет.

In [10]:
# описание числовых данных
toxic_comments.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Колонка `Unnamed: 0` похожа на колонку с индексом, но в ней есть пропуски в нумерации (количество строк в датасете - 159292, а максимальное значение в `Unnamed: 0` - 159450). Для дальнейшего анализа данная колонка не пригодится.

В колонке `toxic` 75% данных представлена 0 (нетоксичные комментарии). Наблюдается дисбаланс классов: токсичных комментариев меньше.

In [11]:
# процентное распределение классов
toxic_comments['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

Токсичных комментариев всего 10% от общего числа данных.

In [12]:
# удалим колонку Unnamed: 0
toxic_comments = toxic_comments.drop('Unnamed: 0', axis=1)

Поскольку ресурсы компьютера (и Google Colab) ограничены, уменьшим выборку до 2 000 комментариев.

In [13]:
toxic_comments_sample = toxic_comments.sample(2000, random_state=RANDOM_STATE).reset_index(drop=True)

<a id='section1.3'></a>
### Предобработка данных

Вместо балансировки классов с помощью SMOTE/RanomOverSampler/RanomUnderSampler будем использовать модель BERT unitary/toxic-bert, которая специально обучена для определения токсичности комментариев. С помощью нее преобразуем тексты комментариев в релевантные для классификации признаки (эмбеддинги).

In [14]:
# используем CUDA
cuda_device = torch.device('cuda')

# инициализируем предобученную модель и токенизатор BERT
model_path = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path).to(cuda_device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Поскольку у модели есть ограничение на 512 токенов, оставим только это количество токенов.

In [15]:
# токенизируем комментарии
tokenized = toxic_comments_sample['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [16]:
# вычисляем максимальное количество токенов (должно получиться 512)
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

max_len

512

Модель BERT работает при условии одинаковой длины токенизированных текстов в корпусе. Будем использовать метод padding, чтобы довести все токенизированные комментарии до длины 512.

In [17]:
# добавим паддинги
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

Добавим attention mask.

In [18]:
# добавим attention mask
attention_mask = np.where(padded != 0, 1, 0)

Переведем токены в векторные представления циклами по 50 шт за раз. Затем объединим их и исходную таблицу с выборкой.

In [19]:
# переводим токены в эмбеддинги по 50 шт за цикл
BATCH_SIZE = 50

embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // BATCH_SIZE)):
    batch = torch.LongTensor(padded[BATCH_SIZE * i:BATCH_SIZE * (i + 1)]).to(cuda_device)
    attention_mask_batch = torch.LongTensor(attention_mask[BATCH_SIZE * i:BATCH_SIZE * (i + 1)]).to(cuda_device)
    with torch.no_grad():
      batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:, 0, :].cpu().numpy())

  0%|          | 0/40 [00:00<?, ?it/s]

In [20]:
# объединяем все созданные эмбеддинги
features = np.concatenate(embeddings)

In [21]:
# переводим эмбеддинги в DataFrame
features_df = pd.DataFrame(features)

In [22]:
# объединяем эмбеддинги с исходной выборкой
toxic_comments_features = toxic_comments_sample.join(features_df)

**Вывод:**

Были загружены и проанализированы данные, в ходе чего выявлен дисбаланс классов. Учитывая ограниченность ресурсов компьютера и Google Colab, из исходных данных случайным образом были выбраны 2 000 комментариев для дальнейшей работы. С помощью предобученной модели BERT unitary/toxic-bert произведена токенизация комментариев, токены переведены в эмбеддинги и объеденены с исходной таблицей с выборкой.

<a id='section2'></a>
## Обучение

Выделим целевую переменную и обучающие признаки. Разобьем данные на тренировочную и тестовую выборки.

In [23]:
# признаки
X = toxic_comments_features.drop(['text', 'toxic'], axis=1)

# целевая переменная
y = toxic_comments_features['toxic']

In [24]:
# разобьем данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=42,
    stratify=y)

print('Размерность входных признаков тренировочной выборки', X_train.shape)
print('Размерность входных признаков обучающей выборки', X_test.shape)
print('Размерность целевого признака тренировочной выборки', y_train.shape)
print('Размерность целевого признака обучающей выборки', y_test.shape)

Размерность входных признаков тренировочной выборки (1500, 768)
Размерность входных признаков обучающей выборки (500, 768)
Размерность целевого признака тренировочной выборки (1500,)
Размерность целевого признака обучающей выборки (500,)


Данные разбились на выборки корректно.

Инициализируем модели, которые будем обучать.

In [25]:
# LogisticRegression
logreg = LogisticRegression(
  random_state=RANDOM_STATE,
  solver='liblinear',
  penalty='l2'
)

# DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

Создадим словари для подбора параметров с помощью RandomizedSearchCV.

In [26]:
# словарь для модели LogisticRegression
logreg_params = {
  'C': range(1,15)
}

# словарь для модели DecisionTreeClassifier
tree_params = {
  'max_depth': range(3,20),
  'max_features': range(3,20),
  'min_samples_split': range(2,20)
}

Создадим скорер с метрикой f1.

In [27]:
def get_f1_score(y_true, y_pred):
    # вычисляем f1_score
    metric = f1_score(y_true, y_pred)
    return metric

# создаём пользовательскую метрику
scorer = make_scorer(get_f1_score)

Создадим функцию, которая будет подбирать гиперпараметры моделей с помощью RandomizedSearchCV, выводить на экран метрику на обучающих данных и лучшую модель.

In [28]:
# функция для подбора гиперпараметров
def rs_func(model, params, n_iter=10, n_jobs=-1):

  # осуществляем подбор гиперпараметров с помощью RandomizedSearchCV
  rs = RandomizedSearchCV(
    model,
    params,
    cv=5,
    scoring=scorer,
    n_iter=n_iter,
    random_state=RANDOM_STATE,
    n_jobs=n_jobs
  )

  rs.fit(X_train, y_train)

  print('Метрика для лучшей модели на кросс-валидации:\n', rs.best_score_)
  print('\nЛучшая модель и её параметры:\n\n', rs.best_estimator_)

  return rs.best_estimator_

По очереди применим функцию к каждой модели.

In [29]:
logreg_best = rs_func(logreg, logreg_params, 3)

Метрика для лучшей модели на кросс-валидации:
 0.9295187903883555

Лучшая модель и её параметры:

 LogisticRegression(C=1, random_state=42, solver='liblinear')


In [30]:
tree_best = rs_func(tree, tree_params)

Метрика для лучшей модели на кросс-валидации:
 0.9079757473709087

Лучшая модель и её параметры:

 DecisionTreeClassifier(max_depth=12, max_features=12, min_samples_split=5,
                       random_state=42)


Среди всех обученных моделей наилучший результат показала LogisticRegression. С помощью нее предскажем токсичность комментариев на тестовых данных и выведем метрику на экран.

In [31]:
y_test_pred = logreg_best.predict(X_test)

print('Метрика на тестовой выборке:', f1_score(y_test, y_test_pred))

Метрика на тестовой выборке: 0.9719626168224299


F1 метрика получилась 0,97 на тестовых данных, что удовлетворяет условиям ТЗ (метрика f1 не меньше 0,75).

<a id='section3'></a>
## Выводы

В ходе анализа данных выявлен дисбаланс классов (10% токсичных комментраиев и 90% нетоксичных).

В связи с ограниченностью ресурсов Google Colab, из датасета взята выборка 2 000 комментариев с распределением классов аналогично исходным данным. С помощью предобученной модели BERT, предназначенной для выявления токсичных комментариев (unitary/toxic-bert), произведена токенизация данных, токены переведены в эмбеддинги и объеденены с таблицей с выборкой.

С помощью RandomizedSearchCV обучены следующие модели и подобраны гиперпараметры к ним: DecisionTreeClassifier и LogisticRegression.

Среди всех обученных моделей для дальнейшего прогноза токсичности комментариев выбрана LogisticRegression со следующими гиперпараметрами: C=1, random_state=42, solver='liblinear'. Метрика f1 на тестовой выборке - 0,97, что удовлетворяет условиям ТЗ.